# Setting up a Ray cluster with SmartSim

## 1. Start the cluster
We set up a SmartSim experiment, which will handle the launch of the Ray cluster.

First we import the relevant modules.

In [2]:
import numpy as np
import time
import argparse
import os

from ray.tune.progress_reporter import JupyterNotebookReporter
import ray
from ray import tune
import ray.util

from smartsim import Experiment
from smartsim.ray import RayCluster

NUM_WORKERS = 3
alloc=None
#alloc=slurm.get_allocation(nodes=1+NUM_WORKERS, time="12:00:00", options={"ntasks": str(1+NUM_WORKERS), "partition": "spider", "C": "V100"})

In [3]:
exp = Experiment("ray-cluster", launcher='slurm')
cluster = RayCluster(name="ray-cluster", run_args={}, path='',
                     launcher='slurm', workers=NUM_WORKERS, alloc=alloc, batch=False, ray_num_cpus=38)

if cluster.batch:
    cluster.head_model.batch_settings._preamble += ["source ~/.bashrc", "conda activate smartsim"]
    if NUM_WORKERS:
        cluster.worker_model.batch_settings._preamble += ["source ~/.bashrc", "conda activate smartsim"]

exp.generate(cluster, overwrite=True)

13:10:43 nid00000 SmartSim[76197] INFO Working in previously created experiment


In [4]:
exp.start(cluster, block=False, summary=False)

13:10:52 nid00000 SmartSim[76197] INFO Ray cluster launched on nodes: ['nid00000', 'nid00003', 'nid00002', 'nid00001']


## 2. Start the ray driver script

In [5]:
ray.util.connect(cluster.head_model.address +":10001")

tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 100).tolist()),
        "log_level": "ERROR",
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=0,
    fail_fast=True,
    log_to_file=True,
)

(pid=100789) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=100789) Instructions for updating:
(pid=100789) non-resource variables are not supported in the long term
(pid=100875) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=100875) Instructions for updating:
(pid=100875) non-resource variables are not supported in the long term
(pid=100817) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_sco

Instructions for updating:
non-resource variables are not supported in the long term


## 3. Stop cluster and release allocation

In [5]:
if alloc:
    slurm.release_allocation(alloc)

In [6]:
exp.stop(cluster)

13:19:07 nid00000 SmartSim[76197] INFO Stopping model workers with job name workers-CBMJ1SJQUDGS
13:19:07 nid00000 SmartSim[76197] INFO Stopping model head with job name head-CBMJ1QLG5SN9


In [9]:

tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
    #    "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 50).tolist()),
        "log_level": "ERROR",
    #    "num_cpus_per_worker": 1,
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=1,
    #fail_fast=True,
    #log_to_file=True,
    progress_reporter = JupyterNotebookReporter(True),
)

<IPython.core.display.HTML object>


(pid=39779) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=39779) Instructions for updating:
(pid=39779) non-resource variables are not supported in the long term
(pid=39781) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=39781) Instructions for updating:
(pid=39781) non-resource variables are not supported in the long term
(pid=113189) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is

<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
(pid=27027) 
(pid=27027) <IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


(pid=27027) 2021-05-25 10:02:22,533	INFO tune.py:549 -- Total run time: 55.35 seconds (55.19 seconds for the tuning loop).


In [10]:
analysis = tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "lr": tune.grid_search([0.001, 0.002, 0.003, 0.004, 0.005]),
    },
    progress_reporter = JupyterNotebookReporter(True)
)

<IPython.core.display.HTML object>


(pid=110180) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110180) Instructions for updating:
(pid=110180) non-resource variables are not supported in the long term
(pid=40586) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=40586) Instructions for updating:
(pid=40586) non-resource variables are not supported in the long term
(pid=110180) 2021-05-25 10:04:22,637	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=42854) WARNING:tensorflow:From /lus/scratch/ar

(pid=27027) 
(pid=27027) <IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


(pid=27027) 2021-05-25 10:04:54,258	INFO tune.py:549 -- Total run time: 34.48 seconds (34.15 seconds for the tuning loop).
(pid=42854) 2021-05-25 10:04:54,221	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=42854) Traceback (most recent call last):
(pid=42854)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=42854)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=42854)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=42854)     return method(__ray_actor, *args, **kwargs)
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1001, in __ray_terminate__
(pid=42854)     ray.actor.exit_actor()
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smart